# Práctica 1 — Dataset 3: Barrios analíticos ↔ Census Tracts (Bronze ➜ Silver)

**Objetivo:** Leer el CSV de relación entre **barrios analíticos** y **census tracts** desde **bronze**,
quedarnos con las columnas necesarias, validar la clave geográfica (`census_tract`) y persistir en **silver** como **Parquet**.

## Requisitos previos (manual)

1. Descarga desde **Moodle**: `2020_census_tracts_to_neighborhoods_20251208.csv`
2. Sube el CSV al bucket **bronze** en la carpeta/prefijo:

   - `sf_neighborhoods_census_tracts/`

In [ ]:
# ============================================================
# 0) Imports + SparkSession
# ============================================================
from pyspark.sql import SparkSession, functions as F, types as T
import os

spark = SparkSession.builder.getOrCreate()
spark.conf.set("spark.sql.session.timeZone", "UTC")

print("Spark version:", spark.version)

In [ ]:
# ============================================================
# 0.1) Configuración MinIO (S3A)
# ============================================================
MINIO_ENDPOINT = os.getenv("MINIO_ENDPOINT", "http://minio:9000")
MINIO_ACCESS_KEY = os.getenv("MINIO_ACCESS_KEY", "minioadmin")
MINIO_SECRET_KEY = os.getenv("MINIO_SECRET_KEY", "minioadmin")
MINIO_USE_SSL = os.getenv("MINIO_USE_SSL", "false").lower() == "true"

hconf = spark._jsc.hadoopConfiguration()
hconf.set("fs.s3a.endpoint", MINIO_ENDPOINT)
hconf.set("fs.s3a.access.key", MINIO_ACCESS_KEY)
hconf.set("fs.s3a.secret.key", MINIO_SECRET_KEY)
hconf.set("fs.s3a.path.style.access", "true")
hconf.set("fs.s3a.connection.ssl.enabled", "true" if MINIO_USE_SSL else "false")
hconf.set("fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider")
hconf.set("fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")

print("MINIO_ENDPOINT =", MINIO_ENDPOINT)

## 1) Lectura desde Bronze (sin inferir esquema)

Leemos desde la carpeta del dataset.

In [ ]:
BRONZE_PATH = "s3a://bronze/sf_neighborhoods_census_tracts/"

df_raw = (
    spark.read
        .option("header", True)
        .option("inferSchema", "false")
        .csv(BRONZE_PATH)
)

df_raw.printSchema()
df_raw.show(5, truncate=False)

## 2) Seleccionar y renombrar columnas en un solo paso

- `neighborhoods_analysis_boundaries` → `analysis_neighborhood`
- `geoid` → `census_tract`

In [ ]:
df = df_raw.select(
    F.col("neighborhoods_analysis_boundaries").alias("analysis_neighborhood"),
    F.col("geoid").alias("census_tract"),
)

df.printSchema()
df.show(10, truncate=False)

## 3) Validación de la clave geográfica `census_tract`

Comprobaciones requeridas:

1. Es de tipo **string**
2. Tiene longitud **11**
3. Conserva correctamente los **ceros a la izquierda**

En las siguientes celdas se realizan las comprobaciones y se muestran métricas para justificar el resultado.

In [ ]:
from pyspark.sql.types import StringType

# 1) Tipo string
is_string = isinstance(df.schema["census_tract"].dataType, StringType)
print("¿census_tract es string?:", is_string)

# 2) Longitud 11
invalid_len = df.filter(F.length("census_tract") != 11).count()
print("Filas con longitud != 11:", invalid_len)

# 3) Ceros a la izquierda:
# Si la columna se hubiese leído como numérica, valores como 06075010101 perderían el 0 inicial.
# Medimos cuántos valores empiezan por '0' y además verificamos que son 11 dígitos.
starts_with_zero = df.filter(F.col("census_tract").startswith("0")).count()
not_11_digits = df.filter(~F.col("census_tract").rlike(r"^\d{11}$")).count()

print("Filas cuyo census_tract empieza por '0':", starts_with_zero)
print("Filas cuyo census_tract NO cumple ^\d{11}$:", not_11_digits)

# Muestra ejemplos (incluye potencialmente valores con 0 inicial)
df.select("census_tract").orderBy("census_tract").show(10, truncate=False)

**Explicación (completa tras ejecutar):**

- **Tipo:** `TODO` (debería ser `true` porque leemos CSV sin inferir esquema, así que Spark lo mantiene como `string`).
- **Longitud:** `TODO` (idealmente 0 filas con longitud distinta de 11).
- **Ceros a la izquierda:** `TODO` (si hay tracts que empiezan por `0` y mantienen longitud 11, los ceros se han preservado).

Si cualquiera de estas comprobaciones falla, revisa la lectura (por ejemplo, `inferSchema` o separadores) y vuelve a cargar.

## 4) Escritura en Silver (Parquet)

Persistimos el resultado en:

- `silver/sf_neighborhoods_census_tracts/`

In [ ]:
SILVER_PATH = "s3a://silver/sf_neighborhoods_census_tracts/"

(
    df.write
      .mode("overwrite")
      .parquet(SILVER_PATH)
)

print("Escritura completada en:", SILVER_PATH)

# (Opcional) Lectura de verificación
df_check = spark.read.parquet(SILVER_PATH)
print("Filas en silver:", df_check.count())
df_check.printSchema()

## 5) Preguntas analíticas

### i) ¿Cuántos census tracts distintos hay?

In [ ]:
n_tracts = df.select("census_tract").distinct().count()
print("Census tracts distintos:", n_tracts)

**Respuesta (completa tras ejecutar):** `TODO`

### ii) ¿Cuántos barrios analíticos distintos hay?

In [ ]:
n_neigh = df.select("analysis_neighborhood").distinct().count()
print("Barrios analíticos distintos:", n_neigh)

**Respuesta (completa tras ejecutar):** `TODO`

### iii) ¿Puede un barrio estar asociado a varios census tracts?

In [ ]:
neigh_stats = (
    df.groupBy("analysis_neighborhood")
      .agg(F.countDistinct("census_tract").alias("n_tracts"))
)

neigh_with_many = neigh_stats.filter(F.col("n_tracts") > 1).count()
max_tracts_per_neigh = neigh_stats.agg(F.max("n_tracts").alias("max_n_tracts")).collect()[0]["max_n_tracts"]

print("Barrios con >1 tract:", neigh_with_many)
print("Máx. nº tracts asociados a un barrio:", max_tracts_per_neigh)

**Respuesta (completa tras ejecutar):**  
- `TODO` (Si existen barrios con más de 1 tract, la respuesta es **sí**.)

### iv) ¿Puede un census tract pertenecer a varios barrios?

In [ ]:
tract_stats = (
    df.groupBy("census_tract")
      .agg(F.countDistinct("analysis_neighborhood").alias("n_neigh"))
)

tracts_with_many = tract_stats.filter(F.col("n_neigh") > 1).count()
max_neigh_per_tract = tract_stats.agg(F.max("n_neigh").alias("max_n_neigh")).collect()[0]["max_n_neigh"]

print("Tracts con >1 barrio:", tracts_with_many)
print("Máx. nº barrios asociados a un tract:", max_neigh_per_tract)

**Respuesta (completa tras ejecutar):**  
- `TODO` (Si existe algún tract con más de 1 barrio, la respuesta es **sí**.)

### v) Justifica brevemente qué tipo de relación existe entre barrios analíticos y census tracts

Usa los resultados de (iii) y (iv) para justificar si la relación es:
- 1 a 1
- 1 a N
- N a 1
- N a M

In [ ]:
# Calculamos indicadores booleanos
can_neigh_have_many_tracts = (neigh_with_many > 0)
can_tract_have_many_neigh = (tracts_with_many > 0)

if can_neigh_have_many_tracts and can_tract_have_many_neigh:
    relation = "N a M (muchos a muchos)"
elif can_neigh_have_many_tracts and not can_tract_have_many_neigh:
    relation = "1 a N (un barrio -> muchos tracts; cada tract -> un barrio)"
elif (not can_neigh_have_many_tracts) and can_tract_have_many_neigh:
    relation = "N a 1 (muchos barrios -> un tract; cada barrio -> un tract)"
else:
    relation = "1 a 1 (uno a uno)"

print("Relación inferida:", relation)

**Respuesta (completa tras ejecutar):**  
- **Tipo de relación:** `TODO`  
- **Justificación:** `TODO` (explica usando el nº de barrios con >1 tract y/o el nº de tracts con >1 barrio)